# Mappeoppgave 2 - Innføring av bompenger i Tromsø: Effekt på bilisters atferd og velferd

Kandidatnr: 75

In [1]:
import pandas as pd
#import dask.dataframe as dd
import numpy as np
import requests
import os
#import chardet
#import time
#import csv
#import gc
#import re

from datetime import datetime, timedelta
#from functools import reduce

import seaborn as sns

from matplotlib import pyplot as plt
#from matplotlib.ticker import MaxNLocator
import matplotlib.ticker as ticker

import matplotlib.pylab as pylab

import sympy as sp
from sympy import *
#from sympy import symbols, Eq, solve, diff

#from bs4 import BeautifulSoup
#from sklearn.linear_model import LinearRegression
#from sklearn.metrics import r2_score

from cycler import cycler

hod_colours = ["#ffa822","#1ac0c6","#ff6150","#30B66A","#B06AFF","#FF21E1"]

# custome plot style
params  = {
"lines.linewidth": 1.5,

"legend.fancybox": "true",

"axes.prop_cycle": cycler('color', hod_colours),
"axes.facecolor": "#2b2b2b",
"axes.axisbelow": "true",
"axes.grid": "true",
"axes.edgecolor": "#2b2b2b",
"axes.linewidth": 0.5,
"axes.labelpad": 0,

"patch.edgecolor": "#2b2b2b",
"patch.linewidth": 0.5,

"grid.linestyle": "--",
"grid.linewidth": 0.5,
"grid.color": "#b8aba7",

"xtick.major.size": 0,
"xtick.minor.size": 0,
"ytick.major.size": 0,
"ytick.minor.size": 0,

"font.family":"monospace",
"font.size":10.0,
"text.color": "#FFE9E3",
"axes.labelcolor": "#b8aba7",
"xtick.color": "#b8aba7",
"ytick.color": "#b8aba7",

"savefig.edgecolor": "#2b2b2b",
"savefig.facecolor": "#2b2b2b",

"figure.subplot.left": 0.08,
"figure.subplot.right": 0.95,
"figure.subplot.bottom": 0.09,
"figure.facecolor": "#2b2b2b"}

pylab.rcParams.update(params)
#print("finish")

In [2]:
### Functions
def get_files(folder:str, file_type:str):
    file_paths = []
    for file in os.listdir(folder):
        if file.endswith(file_type):
            file_paths.append([os.path.join(folder, file), file_type])
    return file_paths

def convert_to_utf8(file_path, delimiter = ",", recoding = 'utf-8'):
    old_name = os.path.splitext(file_path)[0]
    new_file_path = f"{old_name}_{recoding}.csv"
    
    encodings = ["utf-8", "ISO-8859-1", "cp1252", "latin1"]
    success = False

    for encoding in encodings:
        try:
            df = pd.read_csv(file_path, delimiter=delimiter, encoding=encoding)

            success = True
            print(f"Succes with the encoding '{encoding}', recoding to {recoding}, file {new_file_path} now created")
            break

        except UnicodeDecodeError:
            print(f"Failed to read the file with encoding '{encoding}', trying the next one...")

    if not success:
        raise ValueError("None of the attempted encodings were successful in reading the file.")

    df.to_csv(new_file_path, encoding=recoding, index=False, sep = delimiter)
    return new_file_path, delimiter

def read_and_clean_csv(file_path:str, delimiter=",", sep = "",filter_func_list:list=[],):
    df = pd.read_csv(file_path,delimiter=delimiter)
    
    if filter_func_list == []:
        return df
    
    for filter_func in filter_func_list:
        df = filter_func(df)
    if sep == "":
        sep = delimiter
    print(f"sep: {sep}")
    old_name = os.path.splitext(file_path)[0]
    new_file_path = f"{old_name}_cleand.csv"    
    df.to_csv(new_file_path, encoding='utf-8', index=False, sep = sep)
    print(f"Succes cleand, file {new_file_path} now created")
    
    df_clean = pd.read_csv(new_file_path,delimiter=sep)
    return df_clean

In [3]:
# Filter Function:
def filter_keep_columns_day(df, columns_to_keep = ["Navn", "Year", "Month", "Day", "Felt", "Trafikkmengde"]):
    # Keep only the specified columns in the DataFrame
    filtered_df = df[columns_to_keep]
    return filtered_df

def filter_felt(df, value_to_keep = ["1", "2", "Totalt"]):
    # Keep only the specified values in column
    filtered_df = df[(df["Felt"].isin(value_to_keep))]
    return filtered_df

def filter_aggregate_by_month(df):
    df = df.groupby(['Navn', 'Year', 'Month', 'Felt'], as_index=False).agg({'Trafikkmengde': 'sum'})

    return df

def filter_split_date(df):
    df['Dato'] = pd.to_datetime(df['Dato'])

    df['Year'] = df['Dato'].dt.year
    df['Month'] = df['Dato'].dt.month
    df['Day'] = df['Dato'].dt.day

    return df


def filter_sykkel(df):
    df["Sykkel"] = df["Navn"].apply(lambda x: 1 if "sykkel" in str(x).lower() else 0)
    return df

def filter_to_int(df):
    columns_to_int = ["Year", "Month", "Day", "Trafikkmengde"]

    for column in columns_to_int:
        if column in df.columns:
            # Convert numeric strings to integers and non-numeric strings to 0
            df[column] = df[column].apply(lambda x: int(float(x)) if str(x).replace('.', '', 1).isdigit() else 0)
        else:
            print(f"Column '{column}' not found in the DataFrame.")
    return df

In [4]:
### importer dataen
GetFromGit = True # Keep this as True, uless you have the data folder

if GetFromGit:
    url = "https://raw.githubusercontent.com/HodBuri/UIT-oppgaver---SOK1006/main/Mp2/data/trafikk_day_2021_2023_utf-8_cleand.csv"
    df_traffic = pd.read_csv(url)
else:
    DATA_FOLDER = "data/"
    DATA_FOLDER_DATA_TYPE = ".csv"
    file_paths = get_files(DATA_FOLDER, DATA_FOLDER_DATA_TYPE)
    print(file_paths)

In [5]:
if GetFromGit:
    print("Clean file have been collected from github")
else:
    new_file_path, delimiter = convert_to_utf8(file_paths[2][0],delimiter = ";")
    filter_func_list = [filter_split_date, filter_keep_columns_day,filter_felt, filter_sykkel, filter_to_int]
    df_traffic = read_and_clean_csv(new_file_path, delimiter=delimiter, filter_func_list = filter_func_list, sep=",")
    #df_traffic

Clean file have been collected from github


In [6]:
if GetFromGit:
    url = "https://raw.githubusercontent.com/HodBuri/UIT-oppgaver---SOK1006/main/Mp2/data/cost_per_kilometer_2022.csv"
    df_cost_per_km = pd.read_csv(url)
else:
    df_cost_per_km = pd.read_csv(file_paths[0][0])
#df_cost_per_km

In [7]:
if GetFromGit:
    url = "https://raw.githubusercontent.com/HodBuri/UIT-oppgaver---SOK1006/main/Mp2/data/toll_cost.csv"
    df_toll_cost = pd.read_csv(url)
else:
    df_toll_cost = pd.read_csv(file_paths[1][0]) # Note Rushtid = 06:30-09:00 and 15:00-17:00
#df_toll_cost

In [8]:
df_traffic_no_bike = df_traffic[df_traffic["Sykkel"] == 0]
df_traffic_month = df_traffic_no_bike.groupby(["Navn", "Year", "Month", "Felt"]).agg({
    "Trafikkmengde": "sum",
}).reset_index()
#df_traffic_month

In [9]:
df_traffic_yes_bike = df_traffic[df_traffic["Sykkel"] == 1]
df_bike_traffic_month = df_traffic_yes_bike.groupby(["Navn", "Year", "Month", "Felt"]).agg({
    "Trafikkmengde": "sum",
}).reset_index()
#df_bike_traffic_month

In [10]:
df_buss = pd.read_csv("https://raw.githubusercontent.com/uit-sok-1006-v23/uit-sok-1006-v23.github.io/main/data/antall%20p%C3%A5stigende.csv", parse_dates=['date'], dayfirst=True)

In [11]:
#Graph Function
def generate_heatmap(df, year=None, month=None, period=False, line=False, title=None, ):
    if period:
        df_filtered = df[(df['Felt'] == 'Totalt') & (df['Year'].isin(year)) & (df['Month'] <= month)].copy()
        df_filtered['Period'] = df_filtered['Year'].astype(str) + '-' + df_filtered['Month'].astype(str).str.zfill(2)
        pivot_df = df_filtered.pivot_table(values='Trafikkmengde', index='Navn', columns='Period')
    else:
        df_filtered = df[(df['Year'] == year) & (df['Felt'] == 'Totalt')]
        if month is not None:
            df_filtered = df_filtered[df_filtered['Month'] <= month]
        pivot_df = df_filtered.pivot_table(values='Trafikkmengde', index='Navn', columns=['Year', 'Month'])

    pivot_df = pivot_df.fillna(0)

    # Create the heatmap
    plt.figure(figsize=(12, 8))
    sns.heatmap(pivot_df, cmap='viridis')

    if line:
        for i in range(4, pivot_df.columns.shape[0], 4):  
            plt.gca().vlines(i, *plt.gca().get_ylim(), color='white', linestyle='--')

    plt.title(title or "Title")
    plt.ylabel("Navn")
    plt.xlabel("Time (Year-Month)" if period else "Time (Year, Month)")
    plt.show()
    

def plot_total_traffic_with_diff(df, years = [2022, 2023], months=4, column ="Trafikkmengde", condition="Totalt", 
                                 colour= hod_colours, fontsize = 15, text_pos = [0.05,0.01]):

    # Filter and group the data
    filtered_df = df[(df["Felt"] == condition) & (df["Year"].isin(years)) & (df["Month"] <= months)]
    grouped_df = filtered_df.groupby("Year")[column].sum().reset_index()

    # Create a bar graph
    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.bar(grouped_df['Year'].astype(str), grouped_df[column], color=colour)
    ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x)))

    # Calculate the percent difference, should be solo function
    diff = abs(((grouped_df.loc[1, column] - grouped_df.loc[0, column]) / grouped_df.loc[0, column]) * 100)

    # Add data labels
    for bar in bars:
        yval = bar.get_height()
        xval = bar.get_width()
        ax.text(bar.get_x() + xval*text_pos[0], yval*text_pos[1], '{:,.0f}'.format(yval), ha='center', va='bottom', 
                rotation='vertical', color='#2b2b2b', fontsize=fontsize, fontweight='heavy')

    if len(bars) > 1:
        min_bar_index = 0 if bars[0].get_height() < bars[1].get_height() else 1
        max_height = max(bars[0].get_height(), bars[1].get_height())
        
        ax.fill_betweenx([bars[min_bar_index].get_height(), max_height], 
                         bars[min_bar_index].get_x(), 
                         bars[min_bar_index].get_x() + bars[min_bar_index].get_width(), 
                         color=colour[min_bar_index], alpha=0.5)
        
        ax.text(bars[min_bar_index].get_x() + bars[min_bar_index].get_width()/2, 
                bars[min_bar_index].get_height() + (max_height - bars[min_bar_index].get_height()) / 2, 
                f"{diff:.1f}%", ha="center", va="center", color="#ffffff", alpha = 0.7, fontsize=fontsize, fontweight="heavy")

    plt.title(f"Total Traffic for the first {months} months in {years}")
    plt.xlabel("Year")
    plt.ylabel("Total Traffic")
    plt.show()
    
    
# Kvifor!!!!!, This happens when I start messing with other syntaxes, Why CSS and JS!!
# NOTE combine with the plot_total_traffic_with_diff, no point in having two of the same
def plot_total_buss_traffic_with_diff(df, years = [2022, 2023], months=4, column ="Påstigende", 
                                 colour= hod_colours, fontsize = 15, text_pos = [0.05,0.01]):

    # Convert to datetime
    df['date'] = pd.to_datetime(df['date'])

    # Filter and group the data
    filtered_df = df[(df["date"].dt.year.isin(years)) & (df["date"].dt.month <= months)]
    grouped_df = filtered_df.groupby(df["date"].dt.year)[column].sum().reset_index()

    # Create a bar graph
    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.bar(grouped_df['date'].astype(str), grouped_df[column], color=colour)
    ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x)))

    # Calculate the percent difference, should be solo function
    diff = abs(((grouped_df.loc[1, column] - grouped_df.loc[0, column]) / grouped_df.loc[0, column]) * 100)

    # Add data labels
    for bar in bars:
        yval = bar.get_height()
        xval = bar.get_width()
        ax.text(bar.get_x() + xval*text_pos[0], yval*text_pos[1], '{:,.0f}'.format(yval), ha='center', va='bottom', 
                rotation='vertical', color='#2b2b2b', fontsize=fontsize, fontweight='heavy')

    if len(bars) > 1:
        min_bar_index = 0 if bars[0].get_height() < bars[1].get_height() else 1
        max_height = max(bars[0].get_height(), bars[1].get_height())
        
        ax.fill_betweenx([bars[min_bar_index].get_height(), max_height], 
                         bars[min_bar_index].get_x(), 
                         bars[min_bar_index].get_x() + bars[min_bar_index].get_width(), 
                         color=colour, alpha=0.5)
        
        ax.text(bars[min_bar_index].get_x() + bars[min_bar_index].get_width()/2, 
                bars[min_bar_index].get_height() + (max_height - bars[min_bar_index].get_height()) / 2, 
                f"{diff:.1f}%", ha="center", va="center", color="#ffffff", alpha = 0.7, fontsize=fontsize, fontweight="heavy")

    plt.title(f"Total Bus Traffic for the first {months} months in {years}") # Note can add parameter input
    plt.xlabel("Year")
    plt.ylabel("Total Traffic")
    plt.show()

# A simple multi bar plot, horizontal bar orintation
def plot_horizontal_multi_bar(df, x, y, title = None, xlabel = None, ylabel = None, hue = None, orient="h"):
    plt.figure(figsize=(10, 6))
    sns.barplot(y=y, x=x, hue=hue, data=df, orient=orient)
    plt.xlabel(xlabel or x)
    plt.ylabel(ylabel or y)
    plt.title(title or "Title")
    plt.show()

In [12]:
# Economical Functions
def budget(M, px, py, x):
    return (M - px*x)/py

def create_buget_constraints(px1,px2,title:str = None,title_addone:str = ""):
    x = np.linspace(0, 600, 100)

    fig, ax = plt.subplots(figsize=(10,6))

    # Plot the budget line
    ax.plot(x, budget(M_value, px1, py_value, x), label='fossil Car') #px1
    ax.plot(x, budget(M_value, px2, py_value, x), label='fossil Car (toll)') #px2

    ax.set_xlabel('Kilometers Driven')
    ax.set_ylabel('Other Goods')
    ax.set_title(title or "Budget Lines" + title_addone)
    ax.set_ylim(0,1750)
    ax.legend()

    return fig, ax

def create_consumer_pref(px1,px2,title:str = None,title_addone:str = ""):
    # Sympy variables
    x_1, x_2, a, b, U_0, p_1, p_2, m = sp.symbols("x_1 x_2 a b U_0 p_1 p_2 m", real=True, positive=True)

    # Utility function
    U = x_1**a * x_2**b

    # Indifference curve
    x_2_ind_sol = sp.solve(U - U_0, x_2)[0]
    indiff_x_2 = sp.lambdify((U_0, a, b, x_1), x_2_ind_sol)

    # Budget constraint
    budget_constraint = p_1 * x_1 + p_2 * x_2

    # Marginal rate of substitution (MRS)
    MRS = sp.diff(U, x_1) / sp.diff(U, x_2)

    # Solve MRS for x_2 and substitute in budget constraint
    x_2_rel = sp.solve(MRS - p_1 / p_2, x_2)[0]
    x_1_eq = sp.solve(budget_constraint.subs(x_2, x_2_rel) - m, x_1)[0]
    x_2_eq = x_2_rel.subs(x_1, x_1_eq)

    # Maximum utility
    U_max = U.subs([(x_1, x_1_eq), (x_2, x_2_eq)])

    # Parameters
    params1 = [(a, driving_weight), (b, other_weight), (m, M_value), (p_1, px1), (p_2, py_value)]
    params2 = [(a, driving_weight), (b, other_weight), (m, M_value), (p_1, px2), (p_2, py_value)]

    # Calculate max utility and equilibrium quantities
    max_utility = []
    equilibrium_x_1 = []
    equilibrium_x_2 = []
    for params in [params1, params2]:
        max_utility.append(round(U_max.subs(params),2))
        equilibrium_x_1.append(round(x_1_eq.subs(params),2))
        equilibrium_x_2.append(round(x_2_eq.subs(params),2))

    # Plotting
    fig, ax = plt.subplots(figsize=(10,6))
    x = np.linspace(0.1, 1000, 1000)

    # Plot nO toll
    ax.plot(x, indiff_x_2(max_utility[0],driving_weight,other_weight,x), label="Utility max before toll")
    ax.plot(x, budget(M_value, px1, py_value, x), label='Consumer budget before toll')

    # Plot toll
    ax.plot(x, indiff_x_2(max_utility[1],driving_weight,other_weight,x), label="Utility max after toll")
    ax.plot(x, budget(M_value, px2, py_value, x), label='Consumer budget after toll')

    # Plot lines
    ax.vlines(equilibrium_x_1[0],0,equilibrium_x_2[0], color='white',ls='dashed')
    ax.hlines(equilibrium_x_2[0],0,equilibrium_x_1[0], color='white',ls='dashed')

    ax.vlines(equilibrium_x_1[1],0,equilibrium_x_2[1], color='white',ls='dashed')
    ax.hlines(equilibrium_x_2[1],0,equilibrium_x_1[1], color='white',ls='dashed')

    # Labels, title, and stuff
    ax.set_title(title or "Consumer Preferense" + title_addone)

    ax.set_ylabel("$x_2$", loc="top", rotation=0)
    ax.set_xlabel("$x_1$", loc="right")

    ax.set_xlim(0,1000)
    ax.set_ylim(250,1750)
    ax.spines["top"].set_color("none")
    ax.spines["right"].set_color("none")

    # Legend
    ax.legend()

    # X axis text
    txt = "km"
    fig.text(.5,.01, txt, ha='center')

    return fig, ax

## Kapittel 1 - Innledning

I 2022 blei det planlagt å etablere 15 bomstasjoner i Tromsø for å styrke innbyggerens bidrag til et prosject komunen kaller "Bypakken Tenk Tromsø". Den 5 Januar i 2023 var dem klar til bruk. I Norge har bompenger lenge vært en av de viktigste måtene kommunen finansierer prosjecter som er ment for å forbedre byen, I dette tilfelde utgjør bompengene omtrent 58% av den totale budgettet for bypakken på 6,5 milliarder kronder.

Introduksjonen av bomstatsjoner handler imidlertid ikke bare om å samle inn midler for infrastruktur og utvikling. Det er også for å påvirke bilistenes avgjøleser og trafikkmønsteret i byen, ofte med fokus på å redusere miljøpåvirknigen og fremme mer bærekraftige former for transport, som public transport, sykling og elbiler

For å illustrate dette, la oss lage et eksempel med Elisa, en innbygger i Tromsø. Før bomstasjonene ble introdusert pleide Elisa å kjøte bilen sin til jobb hver dag. Men da bomstasjonene blei introdusert, begynte hun å revurdere transportvalgene sine. Ville det være mer økonomisk å sykle eller ta bussen? heller karnsje det er på tide å investere i en elektrisk bil for å unngå høyere avgifter ved bomstasjonene? Dette er spørsmål mange mest sansynelig viss stilte seg selv når bomstasjonene kom.

I de neste kapitlene vil vi ta en dypere titt på trafikkbildet i Tromsø, og hvordan dette påvirket de økonomiske avgjørelsene til byens bilister

## Kapittel 2 - Trafikkbildet i Tromsø før og etter innføring av bompenger

Tromsø's trafikklandskap har gjennomgått en betydelig transformasjon etter inngøring av bomstasjoner, noe som illustrere konsekvensene av svgifter i transportsektoren. For å bedre forstå de endringene som har skjed, vil vi undersøke dataen samlet fra [Vegvesenet](https://www.vegvesen.no/trafikkdata/start/eksport?datatype=DAY&from=2023-04-19&lat=69.667062921797&lon=18.972969608631402&registrationFrequencies=&to=2023-04-20&trafficType=vehicle&zoom=10). La oss først se på trafikkmengden fra 2021 til 2023 

In [13]:
generate_heatmap(df_traffic_month, year=[2021, 2022, 2023], month=4, period=True, line=True, title='Trafikk pre and after, compearing')

<Figure size 3600x2400 with 2 Axes>

Denne figuren gir oss et detaljert overblikk over trafikkmengden i løpet av de første fire mpnedene mellom de nevte årene (2021 - 2023), fordelt på de forskjellige veiene. Det er en tydelig reduksjon i trafikken i 2023, spesielt tydelig på de mest trafikkerte veiene som Breivika og Gjæverbukta. Selv med 2021s lockdowns, ser vi at det pret hadde mer trafikk, noe som antyder at bilistene sannsyneligvis har tilpasset seg alternative transportmetoder.

In [14]:
plot_total_traffic_with_diff(df_traffic_month,fontsize = 40, text_pos = [0.1,0.01])

<Figure size 3000x1800 with 1 Axes>

Ser vi på den totale trafikkmengden fra 2022 og 2023 (limited til de fire først månedene), observerer vi er klart fall. Trafikken har droppet med 16.1%. Dettte er en signifikat reduksjon, og bekrefter at bomstasjonene her en påvirking på Tromsøs bilister. Så hva med andre transportalternativer? Med en så betydelig endring i biltrafikken, er det rimelig å anta at folk har vurdert alternative transportmetoder.

### Trafikk sykkel

In [15]:
generate_heatmap(df_bike_traffic_month, year=[2022, 2023], month=4, period=True, line=True, title='Trafikk sykkel')

<Figure size 3600x2400 with 2 Axes>

I begynnelsen av 2023 ble bomstasjonene innført, og i denne heatmap-figuren observerer vi en klar økning i sykkeltrafikken, spesielt i Tverrforbindelsen og Sandnessbrua. Det interessante er at vi kan se den motsatte trenden i mellom 2022 og 2023. I 2022 begynnte folk å skykle nermere sommertiden, mens i 2023 var det mest sykkel traffiken i starten av året. Dette kan være på grunn av mange faktorer at vi ser en drop nermere summeren. med bra mye. For eksempel at folk var entusiastiske i starten, men ble lei over tid, noe som kan forklare den sterke trafikken i den første måneden. En anden grunn kan være så lett som dårlig vær.

Når vi zoomer litt ut og ser på totalene, observerer vi en økning på 13.11% av syklister fra 2022 til 2023.

In [16]:
plot_total_traffic_with_diff(df_bike_traffic_month,fontsize = 30, text_pos = [0.1,0.01])

<Figure size 3000x1800 with 1 Axes>

In [17]:
plot_total_buss_traffic_with_diff(df_buss,fontsize = 40, text_pos = [0.1,0.01])

<Figure size 3000x1800 with 1 Axes>

Men sykling er ikke det eneste alternativet til bilkjøring, offentlig transport, spesielt buss, har også sett en betydelig økning. Her ser vi en betydelig økning i påstigninger fra 2022 til 2023, med 35.1%. Det er en økning på litt over 1,3 millioner påstigninger, noe som tydelig indikerer at folk søker etter økonomisk effektive alternativer når prisen for bilkjøring øker. Dette er en igjen en forventet reaksjon

Disse endringene i Tromsøs trafikklandskap illustrerer kraften i økonomiske incentiver og hvordan de kan påvirke folks handlinger. Som vi har sett, har innføringen av bomstasjoner ført til en betydelig reduksjon i biltrafikken, mens alternative transportmetoder, som sykling og bruk av offentlig transport, har opplevd en oppgang.

La oss tenke oss Elisa, en bilfører vi introduserte i det forrige kapittelet. Med innføringen av bomstasjoner har hennes daglige pendlerutine blitt dyrere. Hun har nå valget mellom å absorbere de økte kostnadene, eller å finne mer kostnadseffektive alternativer. Kanskje hun velger å sykle til jobb på solrike dager, eller tar bussen når været ikke tillater det. Hun begynner kanskje å vurdere kostnadene og nytten av hver reise hun tar med bilen.

## Kapittel 3 - Bilisters tilpasning til bompenger

Når drivstoffpriser, vedlikehold, og nå bompenger blir stadig dyrere, hvordan tilpasser bilistene seg? Vi ser nærmere på hvordan en bilist fungerer som en konsument som kan velge mellom to goder: "antall kilometer kjørt" og "andre goder". 

In [18]:
plot_horizontal_multi_bar(df_cost_per_km, y= "Kostnader", x="Cost", hue = "Vehicle Type", title = "Cost comparison between Elbiler og Biler med forbrenningsmotor")

<Figure size 3000x1800 with 1 Axes>

Begynner vi med å se på kostnaden per kilometer, henviser vi til data hentet fra [s.41 i dette dokumentet fra TØI.](https://www.toi.no/getfile.php?mmfileid=74283) Her får vi en oversikt over kostnaden per km kjørt i 2022, for både elbiler og biler med forbrenningsmotor. Ved å inkludere drivstoff, dekk, olje, reparasjoner og kapitalkostnader, er km kostnaden for elbiler på 1.98kr, mens for biler med forbrenningsmotor er den 3.03 kr.

Vi vil deretter undersøke de direkte økonomiske virkningene av bompengeinnføringen. Den følgende tabellen viser bompengekostnaden fordelt på tidspunkt, avtalestatus, kjøretøytype og pris.

In [19]:
df_toll_cost

C:\Users\gameb\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\formatters.py:343: FutureWarning:

In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.



,Time Period,Toll Rate,Vehicle Type,Cost
0,Standard,Fullpris,"Bensin, diesel, ladbar hybrid",12.0
1,Standard,Gyldig avtale og brikke,"Bensin, diesel, ladbar hybrid",9.6
2,Standard,Fullpris,Nullutslippskjøretøy,12.0
3,Standard,Gyldig avtale og brikke,Nullutslippskjøretøy,4.8
4,Rushtidsavgift,Fullpris,"Bensin, diesel, ladbar hybrid",36.0
5,Rushtidsavgift,Gyldig avtale og brikke,"Bensin, diesel, ladbar hybrid",28.8
6,Rushtidsavgift,Fullpris,Nullutslippskjøretøy,36.0
7,Rushtidsavgift,Gyldig avtale og brikke,Nullutslippskjøretøy,14.4


Ser vi nærmere på disse dataene, er det tydelig at fullpriser, uavhengig av kjøretøyets utslippsnivå, er de samme både i og utenfor rushtid. Imidlertid er det betydelige rabatter å oppnå ved å skaffe seg en gyldig avtale og brikke.

Ifølge [Nordlys](https://www.nordlys.no/bompenge-snittet-okte-i-mars-de-viktigste-tallene-gar-ned/s/5-34-1776123), er den gjennomsnittlige bompengeregningen omtrent 350 kr per måned i Tromsø. Bemerk at disse dataene ikke er tilgjengelige for offentligheten, og fra personlige samtaler med innbyggere både på øya og i utkanten, har jeg forstått at den faktiske summen ofte ligger nærmere 500 til 700kr. Imidlertid vil vi for nå benytte [Nordlys's tall](https://www.nordlys.no/bompenge-snittet-okte-i-mars-de-viktigste-tallene-gar-ned/s/5-34-1776123).

Vi vil også inkludere data fra [Statistisk sentralbyrå (SSB) - tabell 11418](https://www.ssb.no/en/statbank/table/11418/tableViewLayout1/), som viser at gjennomsnittsinntekten per måned er 53 150kr på tvers av alle sektorer. Videre viser [SSBs tabell 12575](https://www.ssb.no/statbank/table/12575/tableViewLayout1/) at gjennomsnittlig kjørelengde i 2022 var 11 927km for alle kjøretøy, men 11 097km for personbiler, per år.

Med disse dataene kan vi gjøre noen kalkulasjoner og antagelser. For eksempel kan vi dele de årlige kjørelengdene med 12 for å få en gjennomsnittlig kjørelengde per måned

11 097 km / 12 months = 924.75 km/month

Som vi kan se her er 924.75km per måned

Nå, med alle disse dataene samlet, kan vi begynne å dykke dypere inn i analysen. Ved å se på forbrukerbudsjettlinjer og tilpasning til prisendringer, kan vi begynne å forstå hvordan disse endringene i kostnadene påvirker den gjennomsnittlige bilist. Følgende grafer vil illustrere disse poengene ytterligere.

In [20]:
# Given data
avarage_toll_cost_km = round(350/924.75,2) # avarage toll price per mnd / avarage km driven per mnd
montly_income =  53_150 # monthly income
M_value = round(montly_income/31,2) # daily budget
px_value_electric = 1.98  # cost per km for electric car
px_value_electric_toll = px_value_electric + avarage_toll_cost_km  # cost per km for electric car (including toll)
px_value_fossil = 3.03 # cost per km for fossil fuel car
px_value_fossil_toll = px_value_fossil + avarage_toll_cost_km  # cost per km for fossil fuel car (including toll)
px_value_avarage = (px_value_electric + px_value_fossil)/2 # cost per km for fossil fuel car
px_value_avarage_toll = px_value_avarage + avarage_toll_cost_km  # cost per km for fossil fuel car (including toll)
py_value = 1  # cost of other goods

driving_weight = 0.5 # an assumption on how important driving is
other_weight =  0.5 # an assumption on how important other products are

In [21]:
fig, ax = create_buget_constraints(px_value_fossil,px_value_fossil_toll,title_addone = ' (fossil)') #Note Look into the function, there is an errror with km, if time, graph looks correct

<Figure size 3000x1800 with 1 Axes>

In [22]:
fig, ax = create_buget_constraints(px_value_electric,px_value_electric_toll,title_addone = ' (electric)') #Note Look into the function, there is an errror with km, if time, graph looks correct

<Figure size 3000x1800 with 1 Axes>

Her har vi to figurer som vider budsjettkurver, en for elbiler og en for fossil og hybrid biler. Disse linjene representerer de ulike kombinasjoner av "Kilometers Driven" og "Other Goods" en bilist har råd til gitt en bestemt inntekt. Den negative skråningen på disse linjene indikerer at det finnes et trade off mellom disse to godene, det å kjøre mer innebærer at man må gi avkall på noen av de "Other Goods".

Budsjettkurven for fossilbiler har en brattere skråning enn for elbiler. Dette indikerer at kjøreavstanden en fossilbilist kan tilbakelegge for en gitt mengde "Other Goods" er lavere enn for en elbilist. Dette skyldes primært de høyere driftskostnadene for fossilbiler, som drivstoff, vedlikehold, reparasjon og kapitalkostnader.

Etter innføringen av bompenger skifter begge budsjettkurver nedover. Dette illustrerer den økonomiske belastningen som bompenger har lagt på bilistene. På grunn av bompengene har bilistene nå mindre av sin inntekt tilgjengelig for enten "Kilometers Driven" eller "Other Goods". Bompengene reduserer derfor kjøpekraften til bilistene.

In [23]:
fig, ax = create_consumer_pref(px_value_electric,px_value_electric_toll,title_addone = ' (electric)') #Note Look into the function, there is an errror with km, if time, graph looks correct

<Figure size 3000x1800 with 1 Axes>

In [24]:
fig, ax = create_consumer_pref(px_value_fossil,px_value_fossil_toll,title_addone = ' (fossil)') #Note Look into the function, there is an errror with km, if time, graph looks correct

<Figure size 3000x1800 with 1 Axes>

Note at i våres undersøkelse her av bilisters tilpasning til bompenger, har vi valgt å bruke Cobb-Douglas utility funksjon.

Cobb-Douglas nyttefunksjon har nokka kalt constant elasticity of substitution (CES), som gjor den spesielt nyttig i vår analyse. Dette bettyr at responsen i etterspørselen til en vare i forhold til en endring i prisen på en annden vare er konstant, noe som forenkler analysen av bilistenes tilpasning til prisendringer.

Den også gire oss muligheten til å forestille oss bilistenes etterspørsel etter antall kilometer kjørt og andre goder. Dette gjenspeiler tanken om at disse godene konsumeres i bestemte forhold, en antagelse som kan være realistisk for mange bilister.

Når vi sammenligner Cobb douglas nyttefunksjon med Stone Geary nyttefunksjon, finner vi at Cobb Douglas modellen gir en mer fleksibel tilnærming. Stone Geary utility funksjonen antar for eksempel at det er en minimum mengde av hver vare som forbrukeren må konsumere, noe som ikke alltid er tilfelle for bilister. Cobb-Douglas nyttefunksjon er mer tilpassbar og gir derfor et mer nøyaktig bilde av bilistenes tilpasning til innføringen av bompenger.

For å forstå fordelen Cobb-Douglas har over Stone-Geary, la oss ta for oss Elisa igjen. Hun kjører daglig til jobben og har nå med innføringen av bompenger fått et valg å gjøre. Hun kan velge å fortsette å kjøre, ta bussen eller kanskje sykle. I stone geary nyttefunksjon antar vi at det finnes et minimumsbehov for kjøring kanskje 5 kilometer hver dag. Men i virkeligheten, er det ikke nødvendigvis slik for alle bilister. Elisa kan velge å helt droppe bilkjøringen, og dermed ender vi opp med en feilrepresentasjon i Stone Geary modellen.

Cobb-Douglas nyttefunksjonen gir oss derimot større fleksibilitet og realisme i analysen. Den antar ikke et minimumsforbruk, og er dermed mer egnet for å modellere Elisas og andre bilisters tilpasning til bompenger.

De to indifferent kurvene, tegnet over budsjettlinjene i begge grafene, gir oss et visuelt bilde av hvordan bilistene tilpasser seg innføringen av bompenger. For å gi dette mer kontekst, la oss først forklare hva disse kurvene representerer.

Indifferent kurvene representerer forskjellige kombinasjoner av de to godene vi har "Kilometers Driven" (x) og "Other Goods" (y), som gir forbrukeren det samme nivået av tilfredshet, så med andre ord så er de indiffernet på hvor de er i kurven. Punktet der budget linjen berører indifferent kurven er der forbrukeren maksimerer sin nytte gitt sine budget begrensninger.

På den første grafen, som representerer elbiler, kan vi se at indifferent kurvene flytter seg nedover etter innføringen av bompenger. Dette betyr at bilistene nå får mindre nytte for samme mengde utgifter. så for å oppnå samme nivå av nytte må forbrukeren bruke mer penger.

Den samme observasjon kan bli sett for den andre grafen som representerer biler med forbrenningsmotorer og hybrid biler. I dette tilfellet er reduksjonen i nytte mer obvious med den dypere slopen, reflectert av de høyere kostnadene forbundet med kjøring av biler med forbrenningsmotorer.

Hva gjør så bilistene for å tilpasse seg disse endringene?

La oss se på Elisa igjen. Etter at bompenger blei introdusert, opplevde Elisa at hennes budget linje skiftet nedover, noe som betyr at hun nå får mindre nytte for pengene hun bruker på å kjøre. For å maksimere sin nytte, må Elisa enten redusere antall kilometer hun kjører, minimere utgiftene på andre goder, eller en kombinasjon av begge. Noe som vi igjen kunne se i kapitel two når vi så over endiringen i trafikken, vi kunne klart se at bilistene fant subssitution for den økente kosten for kjøring, med en drop i trafikk med 16.1% og en økining i sykkling og buss påstigninger med 13.1% og 35.1%.

## Kapittel 4 - Konklusjon

So i konklusjon, vi har sett på hvordan trafikken så ut før og etter bompengene blei introdusert, og hvordan trafikken ble påvirket. Vi så en kraftig drop i trafikk med 16.1%, og vi så en trend på økning i andre transport milder slik som sykkling og buss. Og vi så på hvordan disse økonomiske pressures endrer bilister slik som Elisa, der dem må finne en måtte å balansere needed av transport og den økede kosten.

Vi har også sett mer spesielt på elbiler vs fossil biler, der vi så at ja det har vært en shift i begge, men det er klart bedre å kjøre elbiler. Noe som vi sa i kapitel 1:

"Introduksjonen av bomstatsjoner handler imidlertid ikke bare om å samle inn midler for infrastruktur og utvikling. Det er også for å påvirke bilistenes avgjøleser og trafikkmønsteret i byen, ofte med fokus på å redusere miljøpåvirknigen og fremme mer bærekraftige former for transport, som public transport, sykling og elbiler"

Vi ser at dataen og analyzen vi har gått over passer dette. Bomstasjoner være et effektivt verktøy for å styre trafikkmønsteret i byen og fremme mer bærekraftige transportformer. Men det er også viktig å merke seg at innføringen av bompenger legger en ekstra økonomisk byrde på bilistene. Hver enkelt bilist, som Elisa, har måttet tilpasse seg, og det er dette som til sist driver endringene vi har analysert. Så selv om bomstasjonene kan være kontroversielle og kreve tilpasning, ser det ut til at de bidrar til en langsiktig positiv effekt på byens transportmønstre og bærekraft.